In [19]:
import akshare as ak

import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datetime import datetime

In [2]:
STOCK_CODES = ["600519", "000858", "600938", "000333", "601088", "300866", "600900", "600036"]
STOCK_SYMBOLS_1 = [code + ".SH" if code.startswith("6") else code + ".SZ" for code in STOCK_CODES ]
STOCK_SYMBOLS_2 = ['sh' + code if code.startswith("6") else 'sz' + code for code in STOCK_CODES]

In [3]:
investment_dict = {
    "stock_code": ["600519", "000858", "600938", "000333", "601088", "300866", "600900"],
    "cost_per_share": [1482.8976, 139.3360, 27.4303, 76.5841, 42.1904, 94.5264, 27.1260], 
    "shares": [100, 700, 2600, 700, 1000, 400, 1200]
}

investment_df = pd.DataFrame(investment_dict)
investment_df

,stock_code,cost_per_share,shares
0,600519,1482.8976,100
1,000858,139.3360,700
2,600938,27.4303,2600
3,000333,76.5841,700
4,601088,42.1904,1000
5,300866,94.5264,400
6,600900,27.1260,1200


In [6]:
dfs = []
season_gap = 2 # the most recent season data

for symbol in tqdm(STOCK_SYMBOLS_1):
    # load the eps and roe data
    eps_roe_df = ak.stock_financial_analysis_indicator_em(symbol=symbol, indicator="按报告期")
    eps_roe_df = eps_roe_df[["REPORT_DATE", "REPORT_TYPE", "REPORT_DATE_NAME", 
                            "EPSJB", "BPS", "ROEJQ"]]

    # rename the columns
    eps_roe_df.columns = ["date", "report_type", "report_date_type", "eps", "bps", "roe"]

    # calculate eps ttm
    eps_roe_df['eps_season'] = eps_roe_df['eps'].diff(-1)
    eps_roe_df['eps_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                        eps_roe_df['eps'], eps_roe_df['eps_season'])
    eps_roe_df['eps_ttm'] = eps_roe_df['eps_season'].rolling(4).sum().shift(-3)

    # calculate roe ttm
    eps_roe_df['roe_season'] = eps_roe_df['roe'].diff(-1)
    eps_roe_df['roe_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                        eps_roe_df['roe'], eps_roe_df['roe_season'])
    eps_roe_df['roe_ttm'] = eps_roe_df['roe_season'].rolling(4).sum().shift(-3)

    eps_roe_df = eps_roe_df.iloc[season_gap]
    eps_roe_df = eps_roe_df[['date', 'report_type', 'report_date_type', 'eps_ttm', 'bps', 'roe_ttm']]
    eps_roe_df['stock_code'] = symbol[:6]
    dfs.append(eps_roe_df)
    
stock_df = pd.DataFrame(dfs)
portfolio_df = pd.merge(investment_df, stock_df, on='stock_code', how='left', validate="1:1")

# calculate portfolio metrics
portfolio_df = portfolio_df.eval("cost = cost_per_share * shares")
portfolio_df = portfolio_df.eval("porfolio_earning = eps_ttm * shares")
portfolio_df = portfolio_df.eval("porfolio_net_asset = bps * shares")

portfolio_df.loc["portfolio", "cost"] = portfolio_df["cost"].sum()
portfolio_df.loc["portfolio", "porfolio_earning"] = portfolio_df["porfolio_earning"].sum()
portfolio_df.loc["portfolio", "porfolio_net_asset"] = portfolio_df["porfolio_net_asset"].sum()

portfolio_df = portfolio_df.eval("earning_yield = porfolio_earning / cost")
portfolio_df = portfolio_df.eval("net_asset_yield = porfolio_net_asset / cost")

portfolio_df.loc["portfolio", "stock_code"] = "portfolio" + "_season_gap_" + str(season_gap)
portfolio_df.loc["portfolio", "date"] = portfolio_df.iloc[0, 3]
portfolio_df.loc["portfolio", "report_type"] = portfolio_df.iloc[0, 4]

portfolio_df

100%|██████████| 8/8 [00:04<00:00,  1.67it/s]


,stock_code,cost_per_share,shares,date,report_type,report_date_type,eps_ttm,bps,roe_ttm,cost,porfolio_earning,porfolio_net_asset,earning_yield,net_asset_yield
0,600519,1482.8976,100.0,2025-03-31 00:00:00,一季报,2025一季报,70.8600,205.666530,36.37,148289.76,7086.00,20566.653012,0.047785,0.138692
1,000858,139.3360,700.0,2025-03-31 00:00:00,一季报,2025一季报,8.4162,38.166025,23.63,97535.20,5891.34,26716.217430,0.060402,0.273914
2,600938,27.4303,2600.0,2025-03-31 00:00:00,一季报,2025一季报,2.8300,16.491158,18.35,71318.78,7358.00,42877.011846,0.103171,0.601202
3,000333,76.5841,700.0,2025-03-31 00:00:00,一季报,2025一季报,5.7700,30.001774,21.48,53608.87,4039.00,21001.241848,0.075342,0.391749
4,601088,42.1904,1000.0,2025-03-31 00:00:00,一季报,2025一季报,2.8210,21.799007,13.30,42190.40,2821.00,21799.006719,0.066864,0.516682
5,300866,94.5264,400.0,2025-03-31 00:00:00,一季报,2025一季报,4.3388,17.613782,26.48,37810.56,1735.52,7045.512819,0.045900,0.186337
6,600900,27.1260,1200.0,2025-03-31 00:00:00,一季报,2025一季报,1.3776,8.820215,16.19,32551.20,1653.12,10584.258117,0.050785,0.325157
portfolio,portfolio_season_gap_2,NaN,NaN,2025-03-31 00:00:00,一季报,NaN,NaN,NaN,NaN,483304.77,30583.98,150589.901792,0.063281,0.311584


In [18]:
portfolio_dfs = []

for season_gap in tqdm(range(12)): # season gap = 0 is the most recent season
    dfs = []
    for symbol in STOCK_SYMBOLS_1:
        # load the eps and roe data
        eps_roe_df = ak.stock_financial_analysis_indicator_em(symbol=symbol, indicator="按报告期")
        eps_roe_df = eps_roe_df[["REPORT_DATE", "REPORT_TYPE", "REPORT_DATE_NAME", 
                                "EPSJB", "BPS", "ROEJQ"]]

        # rename the columns
        eps_roe_df.columns = ["date", "report_type", "report_date_type", "eps", "bps", "roe"]

        # calculate eps ttm
        eps_roe_df['eps_season'] = eps_roe_df['eps'].diff(-1)
        eps_roe_df['eps_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                            eps_roe_df['eps'], eps_roe_df['eps_season'])
        eps_roe_df['eps_ttm'] = eps_roe_df['eps_season'].rolling(4).sum().shift(-3)

        # calculate roe ttm
        eps_roe_df['roe_season'] = eps_roe_df['roe'].diff(-1)
        eps_roe_df['roe_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                            eps_roe_df['roe'], eps_roe_df['roe_season'])
        eps_roe_df['roe_ttm'] = eps_roe_df['roe_season'].rolling(4).sum().shift(-3)

        eps_roe_df = eps_roe_df.iloc[season_gap]
        eps_roe_df = eps_roe_df[['date', 'report_type', 'report_date_type', 'eps_ttm', 'bps', 'roe_ttm']]
        eps_roe_df['stock_code'] = symbol[:6]
        dfs.append(eps_roe_df)
        
    stock_df = pd.DataFrame(dfs)
    portfolio_df = pd.merge(investment_df, stock_df, on='stock_code', how='left', validate="1:1")

    # calculate portfolio metrics
    portfolio_df = portfolio_df.eval("cost = cost_per_share * shares")
    portfolio_df = portfolio_df.eval("porfolio_earning = eps_ttm * shares")
    portfolio_df = portfolio_df.eval("porfolio_net_asset = bps * shares")

    portfolio_df.loc["portfolio", "cost"] = portfolio_df["cost"].sum()
    portfolio_df.loc["portfolio", "porfolio_earning"] = portfolio_df["porfolio_earning"].sum()
    portfolio_df.loc["portfolio", "porfolio_net_asset"] = portfolio_df["porfolio_net_asset"].sum()

    portfolio_df = portfolio_df.eval("earning_yield = porfolio_earning / cost")
    portfolio_df = portfolio_df.eval("net_asset_yield = porfolio_net_asset / cost")

    portfolio_df.loc["portfolio", "stock_code"] = "portfolio" + "_season_gap_" + str(season_gap)
    portfolio_df.loc["portfolio", "date"] = portfolio_df.iloc[0, 3]
    portfolio_df.loc["portfolio", "report_type"] = portfolio_df.iloc[0, 4]

    portfolio_dfs.append(portfolio_df)

portfolio_dfs = pd.concat(portfolio_dfs)
portfolio_dfs.loc["portfolio"]

100%|██████████| 12/12 [00:50<00:00,  4.21s/it]


,stock_code,cost_per_share,shares,date,report_type,report_date_type,eps_ttm,bps,roe_ttm,cost,porfolio_earning,porfolio_net_asset,earning_yield,net_asset_yield
portfolio,portfolio_season_gap_0,NaN,NaN,2025-09-30 00:00:00,三季报,NaN,NaN,NaN,NaN,483304.77,29399.08,148456.645485,0.060829,0.307170
portfolio,portfolio_season_gap_1,NaN,NaN,2025-06-30 00:00:00,中报,NaN,NaN,NaN,NaN,483304.77,30310.01,144294.915245,0.062714,0.298559
portfolio,portfolio_season_gap_2,NaN,NaN,2025-03-31 00:00:00,一季报,NaN,NaN,NaN,NaN,483304.77,30583.98,150589.901792,0.063281,0.311584
portfolio,portfolio_season_gap_3,NaN,NaN,2024-12-31 00:00:00,年报,NaN,NaN,NaN,NaN,483304.77,30100.62,141843.440828,0.062281,0.293487
portfolio,portfolio_season_gap_4,NaN,NaN,2024-09-30 00:00:00,三季报,NaN,NaN,NaN,NaN,483304.77,29575.32,139389.110568,0.061194,0.288408
portfolio,portfolio_season_gap_5,NaN,NaN,2024-06-30 00:00:00,中报,NaN,NaN,NaN,NaN,483304.77,28829.21,132446.843010,0.059650,0.274044
portfolio,portfolio_season_gap_6,NaN,NaN,2024-03-31 00:00:00,一季报,NaN,NaN,NaN,NaN,483304.77,27951.32,140692.756684,0.057834,0.291106
portfolio,portfolio_season_gap_7,NaN,NaN,2023-12-31 00:00:00,年报,NaN,NaN,NaN,NaN,483304.77,27170.78,131512.939623,0.056219,0.272112
portfolio,portfolio_season_gap_8,NaN,NaN,2023-09-30 00:00:00,三季报,NaN,NaN,NaN,NaN,483304.77,27153.34,127192.930220,0.056183,0.263173
portfolio,portfolio_season_gap_9,NaN,NaN,2023-06-30 00:00:00,中报,NaN,NaN,NaN,NaN,483304.77,26704.54,121657.245330,0.055254,0.251720


In [20]:
today = datetime.today().strftime('%Y%m%d')

portfolio_dfs.to_csv(f"../data/processed/portfolio_intrinsic_value_over_seasons_{today}.csv")